# Gathering data
- WeRateDogs twitter archive

In [4]:
import requests
import os
import json
import pandas as pd

In [5]:
# Reading 'twitter_archive_enhanced.csv' file into a DataFrame
twitter_archive_df = pd.read_csv('twitter-archive-enhanced.csv')

- Downloading image predictions dataset programaticaly

In [3]:
# Downloading 'image-predictions.tsv' programaticaly

folder_name = 'image_predictions'
if not os.path.exists(folder_name) :
    os.makedirs(folder_name)

url = 'https://d17h27t6h515a5.cloudfront.net/topher/2017/August/599fd2ad_image-predictions/image-predictions.tsv'  
response = requests.get(url) 


<Response [200]>

In [5]:
# Opening file

with open (os.path.join(folder_name, url.split('/')[-1]), mode = 'wb') as file:
    file.write(response.content)

In [6]:
# Reading the file into a DataFrame

image_df = pd.read_csv('image_predictions/image-predictions.tsv',sep = '\t')

- Getting Tweeter API dataset

In [ ]:
import tweepy
from tweepy import OAuthHandler
import json
from timeit import default_timer as timer

# Query Twitter API for each tweet in the Twitter archive and save JSON in a text file
# These are hidden to comply with Twitter's API terms and conditions
consumer_key = 'HIDDEN'
consumer_secret = 'HIDDEN'
access_token = 'HIDDEN'
access_secret = 'HIDDEN'

auth = OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_secret)

api = tweepy.API(auth, wait_on_rate_limit=True)

# NOTE TO STUDENT WITH MOBILE VERIFICATION ISSUES:
# df_1 is a DataFrame with the twitter_archive_enhanced.csv file. You may have to
# change line 17 to match the name of your DataFrame with twitter_archive_enhanced.csv
# NOTE TO REVIEWER: this student had mobile verification issues so the following
# Twitter API code was sent to this student from a Udacity instructor
# Tweet IDs for which to gather additional data via Twitter's API
tweet_ids = twitter_archive_df.tweet_id.values
len(tweet_ids)

# Query Twitter's API for JSON data for each tweet ID in the Twitter archive
count = 0
fails_dict = {}
start = timer()
# Save each tweet's returned JSON as a new line in a .txt file
with open('tweet_json.txt', 'w') as outfile:
    # This loop will likely take 20-30 minutes to run because of Twitter's rate limit
    for tweet_id in tweet_ids:
        count += 1
        print(str(count) + ": " + str(tweet_id))
        try:
            tweet = api.get_status(tweet_id, tweet_mode='extended')
            print("Success")
            json.dump(tweet._json, outfile)
            outfile.write('\n')
        except tweepy.TweepError as e:
            print("Fail")
            fails_dict[tweet_id] = e
            pass
end = timer()
print(end - start)
print(fails_dict)

In [7]:
# Reading the 'tweet_json' data into a DataFrame

df_list=[]

with open('tweet_json.txt') as file:
    
    for line in file:
        
        readtweet = file.readline()
        tweetdict = json.loads(readtweet) 
        
        tweet_id = tweetdict['id']
        retweet_count= tweetdict['retweet_count']
        fav_count= tweetdict['favorite_count']
        
        
        # Append to list of dictionaries
        df_list.append({'tweet_id': tweet_id,
                        'retweet_count': retweet_count,
                        'favorite_count': fav_count})
        
api_df = pd.DataFrame(df_list, columns = ['tweet_id', 'retweet_count', 'favorite_count'])


# Assessing Data
### Issues to clean
Note : issues are preceded with a number to indicate its priority.
#### Quality issues:
- (3) "twitter_archive_df" has 2356 rows while the "image_predictions_df" has only 2075 rows, probably due to retweets and missing photos.
- (2) "twitter_archive_df" contains 181 retweets and 78 replies which not needed.
- (5) there are many names of dogs which are clearly incorrect such as 'a', 'actually', 'all'
- (6) strange unexpected values in rating_denominator
- (7) source data column is not clear
- (8) timestamp column has not necessary +0000
- (9) change timestamp data type to datetime
- (10) consistent names and logical order to column data as final step in cleaning

#### Tidiness issues:
- (1) there are 3 dataframes, only 1 dataframe should suffice to incorporate the data we need.
- (4) there are 4 columns for doggo, floffer, pupper, and puppo in the "twitter_archive_df" but this is same categorical data that would be better in only 1 column.


### Exploring data
Using [.head(), .info(), .shape() and etc.]

In [8]:
# Viewing tweeter_archive_dataframe snapshot
twitter_archive_df.head(3)

tweet_id  in_reply_to_status_id  in_reply_to_user_id  \
0  892420643555336193                    NaN                  NaN   
1  892177421306343426                    NaN                  NaN   
2  891815181378084864                    NaN                  NaN   

                   timestamp  \
0  2017-08-01 16:23:56 +0000   
1  2017-08-01 00:17:27 +0000   
2  2017-07-31 00:18:03 +0000   

                                              source  \
0  <a href="http://twitter.com/download/iphone" r...   
1  <a href="http://twitter.com/download/iphone" r...   
2  <a href="http://twitter.com/download/iphone" r...   

                                                text  retweeted_status_id  \
0  This is Phineas. He's a mystical boy. Only eve...                  NaN   
1  This is Tilly. She's just checking pup on you....                  NaN   
2  This is Archie. He is a rare Norwegian Pouncin...                  NaN   

   retweeted_status_user_id retweeted_status_timestamp  \
0                       NaN                        NaN   
1                       NaN                        NaN   
2                       NaN                        NaN   

                                       expanded_urls  rating_numerator  \
0  https://twitter.com/dog_rates/status/892420643...                13   
1  https://twitter.com/dog_rates/status/892177421...                13   
2  https://twitter.com/dog_rates/status/891815181...                12   

   rating_denominator     name doggo floofer pupper puppo  
0                  10  Phineas  None    None   None  None  
1                  10    Tilly  None    None   None  None  
2                  10   Archie  None    None   None  None

In [11]:
twitter_archive_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2356 entries, 0 to 2355
Data columns (total 17 columns):
tweet_id                      2356 non-null int64
in_reply_to_status_id         78 non-null float64
in_reply_to_user_id           78 non-null float64
timestamp                     2356 non-null object
source                        2356 non-null object
text                          2356 non-null object
retweeted_status_id           181 non-null float64
retweeted_status_user_id      181 non-null float64
retweeted_status_timestamp    181 non-null object
expanded_urls                 2297 non-null object
rating_numerator              2356 non-null int64
rating_denominator            2356 non-null int64
name                          2356 non-null object
doggo                         2356 non-null object
floofer                       2356 non-null object
pupper                        2356 non-null object
puppo                         2356 non-null object
dtypes: float64(4), int64(3), ob

In [12]:
twitter_archive_df.shape

(2356, 17)

In [9]:
# Viewing image_predictions_dataframe snapshot
image_df.head(3)

tweet_id                                          jpg_url  \
0  666020888022790149  https://pbs.twimg.com/media/CT4udn0WwAA0aMy.jpg   
1  666029285002620928  https://pbs.twimg.com/media/CT42GRgUYAA5iDo.jpg   
2  666033412701032449  https://pbs.twimg.com/media/CT4521TWwAEvMyu.jpg   

   img_num                      p1   p1_conf  p1_dog                  p2  \
0        1  Welsh_springer_spaniel  0.465074    True              collie   
1        1                 redbone  0.506826    True  miniature_pinscher   
2        1         German_shepherd  0.596461    True            malinois   

    p2_conf  p2_dog                   p3   p3_conf  p3_dog  
0  0.156665    True    Shetland_sheepdog  0.061428    True  
1  0.074192    True  Rhodesian_ridgeback  0.072010    True  
2  0.138584    True           bloodhound  0.116197    True

In [13]:
image_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2075 entries, 0 to 2074
Data columns (total 12 columns):
tweet_id    2075 non-null int64
jpg_url     2075 non-null object
img_num     2075 non-null int64
p1          2075 non-null object
p1_conf     2075 non-null float64
p1_dog      2075 non-null bool
p2          2075 non-null object
p2_conf     2075 non-null float64
p2_dog      2075 non-null bool
p3          2075 non-null object
p3_conf     2075 non-null float64
p3_dog      2075 non-null bool
dtypes: bool(3), float64(3), int64(2), object(4)
memory usage: 152.1+ KB


In [14]:
image_df.shape

(2075, 12)

In [10]:
# Viewing TweeterAPI_dataframe snapshot
api_df.head(3)

tweet_id  retweet_count  favorite_count
0  892177421306343426           6514           33819
1  891689557279858688           8964           42908
2  891087950875897856           3261           20562

In [15]:
api_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1177 entries, 0 to 1176
Data columns (total 3 columns):
tweet_id          1177 non-null int64
retweet_count     1177 non-null int64
favorite_count    1177 non-null int64
dtypes: int64(3)
memory usage: 27.7 KB


In [17]:
api_df.shape

(1177, 3)

#### 1) Merging the three tables together

In [18]:
# Creat a copy of the original 3 datasets for reference
archive_clean_df = twitter_archive_df.copy()
image_clean_df = image_df.copy()
api_clean_df = api_df.copy()

In [19]:
# Merge the three DataFrames into one master DataFrame, starting first with archive_clean and json_clean
first_archive_master_df = pd.merge(archive_clean_df, api_clean_df, on=['tweet_id','tweet_id'],how='left')

# Merge new master with image_clean
first_archive_master_df = pd.merge(first_archive_master_df, image_clean_df, on='tweet_id',how='left')

In [21]:
first_archive_master_df.shape

(2356, 30)

In [22]:
# Save a copy of the newly created master dataframe, to be a starting point for cleaning
first_archive_master_df.to_csv('first_archive_master.csv',encoding='utf-8',index=False)

In [52]:
# Reading the newly created master dataframe
df = pd.read_csv('first_archive_master.csv')

#### Now all three tables are merged into a single dataframe called (df)

#### 2) Removing retweets and replies

In [29]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2356 entries, 0 to 2355
Data columns (total 30 columns):
tweet_id                      2356 non-null int64
in_reply_to_status_id         78 non-null float64
in_reply_to_user_id           78 non-null float64
timestamp                     2356 non-null object
source                        2356 non-null object
text                          2356 non-null object
retweeted_status_id           181 non-null float64
retweeted_status_user_id      181 non-null float64
retweeted_status_timestamp    181 non-null object
expanded_urls                 2297 non-null object
rating_numerator              2356 non-null int64
rating_denominator            2356 non-null int64
name                          2356 non-null object
doggo                         2356 non-null object
floofer                       2356 non-null object
pupper                        2356 non-null object
puppo                         2356 non-null object
retweet_count                 11

In [53]:
# there are 181 retweets, which can be seen in the 3 columns "retweeted_status_id", "retweeted_status_user_id" and 
# "retweeted_status_timestamp". We keep the rows that are null and remove the retweets.
df = df[df.retweeted_status_id.isnull()]

In [54]:
# delete the columns related to retweets from the master dataframe.
df = df.drop(['retweeted_status_id','retweeted_status_user_id','retweeted_status_timestamp'],axis=1)

In [55]:

# the "in_reply_to_status_id" includes the original tweet_id if there is a reply. There are 78 replies.
# We keep the rows that are null and remove the replies.
df = df[df.in_reply_to_status_id.isnull()]

In [56]:
# delete the columns related to replies from the master dataframe.
df = df.drop(['in_reply_to_status_id','in_reply_to_user_id'],axis=1)

In [36]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2097 entries, 0 to 2355
Data columns (total 25 columns):
tweet_id              2097 non-null int64
timestamp             2097 non-null object
source                2097 non-null object
text                  2097 non-null object
expanded_urls         2094 non-null object
rating_numerator      2097 non-null int64
rating_denominator    2097 non-null int64
name                  2097 non-null object
doggo                 2097 non-null object
floofer               2097 non-null object
pupper                2097 non-null object
puppo                 2097 non-null object
retweet_count         1047 non-null float64
favorite_count        1047 non-null float64
jpg_url               1971 non-null object
img_num               1971 non-null float64
p1                    1971 non-null object
p1_conf               1971 non-null float64
p1_dog                1971 non-null object
p2                    1971 non-null object
p2_conf               1971 non-n

#### 3) Removing tweets that do not have an image

In [57]:
# After deleting the retweets and replies we still have 126 tweets without an image, as this was one of the project's
# requirements we will delete the tweets without an image. This time we will keep the rows with an image.
df = df[df.jpg_url.notnull()]

In [38]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1971 entries, 0 to 2355
Data columns (total 25 columns):
tweet_id              1971 non-null int64
timestamp             1971 non-null object
source                1971 non-null object
text                  1971 non-null object
expanded_urls         1971 non-null object
rating_numerator      1971 non-null int64
rating_denominator    1971 non-null int64
name                  1971 non-null object
doggo                 1971 non-null object
floofer               1971 non-null object
pupper                1971 non-null object
puppo                 1971 non-null object
retweet_count         987 non-null float64
favorite_count        987 non-null float64
jpg_url               1971 non-null object
img_num               1971 non-null float64
p1                    1971 non-null object
p1_conf               1971 non-null float64
p1_dog                1971 non-null object
p2                    1971 non-null object
p2_conf               1971 non-nul

#### 4) Melting the 4 dog stage columns into a single column

In [43]:
# Check how many dog types there are of each
df['doggo'].value_counts()

None     1898
doggo      73
Name: doggo, dtype: int64

In [44]:
df['floofer'].value_counts()

None       1963
floofer       8
Name: floofer, dtype: int64

In [45]:
df['pupper'].value_counts()

None      1762
pupper     209
Name: pupper, dtype: int64

In [46]:
df['puppo'].value_counts()

None     1948
puppo      23
Name: puppo, dtype: int64

In [49]:
df.columns

Index(['tweet_id', 'timestamp', 'source', 'text', 'expanded_urls',
       'rating_numerator', 'rating_denominator', 'name', 'doggo', 'floofer',
       'pupper', 'puppo', 'retweet_count', 'favorite_count', 'jpg_url',
       'img_num', 'p1', 'p1_conf', 'p1_dog', 'p2', 'p2_conf', 'p2_dog', 'p3',
       'p3_conf', 'p3_dog'],
      dtype='object')

In [59]:
# Melt the 4 columns into a column'types' to be deleted and a value'dog_type'column to keep.
# save into a new datafram called (df_melted)
df_melted = pd.melt(df, 
        id_vars = ['tweet_id', 'timestamp', 'source', 'text', 'expanded_urls',
       'rating_numerator', 'rating_denominator', 'name', 'favorite_count', 'retweet_count', 'jpg_url', 'img_num',
       'p1', 'p1_conf', 'p1_dog', 'p2', 'p2_conf', 'p2_dog', 'p3', 'p3_conf',
       'p3_dog'], 
        value_vars = ['doggo', 'floofer', 'pupper', 'puppo'], 
        var_name = 'types', value_name = 'dog_type')


In [72]:
df_melted.head(2)

tweet_id                  timestamp  \
1895  667405339315146752  2015-11-19 18:13:27 +0000   
1894  667435689202614272  2015-11-19 20:14:03 +0000   

                                                 source  \
1895  <a href="http://twitter.com/download/iphone" r...   
1894  <a href="http://twitter.com/download/iphone" r...   

                                                   text  \
1895  This is Biden. Biden just tripped... 7/10 http...   
1894            Ermergerd 12/10 https://t.co/PQni2sjPsm   

                                          expanded_urls  rating_numerator  \
1895  https://twitter.com/dog_rates/status/667405339...                 7   
1894  https://twitter.com/dog_rates/status/667435689...                12   

      rating_denominator   name  favorite_count  retweet_count    ...     \
1895                  10  Biden           489.0          234.0    ...      
1894                  10   None             NaN            NaN    ...      

                 p1   p1_conf p1_dog                  p2   p2_conf p2_dog  \
1895  Saint_Bernard  0.381377   True            Leonberg  0.127998   True   
1894     Rottweiler  0.999091   True  miniature_pinscher  0.000450   True   

                           p3   p3_conf p3_dog  dog_type  
1895         golden_retriever  0.069357   True      None  
1894  black-and-tan_coonhound  0.000157   True      None  

[2 rows x 22 columns]

In [63]:
# Check we still have all our dog types
df_melted.dog_type.value_counts()

None       7571
pupper      209
doggo        73
puppo        23
floofer       8
Name: dog_type, dtype: int64

In [64]:
# Sort the dataframe by dog type so that we can drop the 3 duplicates that don't contain a dog type.
df_melted = df_melted.sort_values('dog_type').drop_duplicates('tweet_id', keep = 'last')

In [69]:
# Delete column 'types'
df_melted = df_melted.drop('types',axis=1)

In [71]:
# Check the dataframe status and the dog type value counts.
df_melted.info()
df_melted.dog_type.value_counts()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1971 entries, 1895 to 6618
Data columns (total 22 columns):
tweet_id              1971 non-null int64
timestamp             1971 non-null object
source                1971 non-null object
text                  1971 non-null object
expanded_urls         1971 non-null object
rating_numerator      1971 non-null int64
rating_denominator    1971 non-null int64
name                  1971 non-null object
favorite_count        987 non-null float64
retweet_count         987 non-null float64
jpg_url               1971 non-null object
img_num               1971 non-null float64
p1                    1971 non-null object
p1_conf               1971 non-null float64
p1_dog                1971 non-null object
p2                    1971 non-null object
p2_conf               1971 non-null float64
p2_dog                1971 non-null object
p3                    1971 non-null object
p3_conf               1971 non-null float64
p3_dog                1971 no

None       1668
pupper      209
doggo        63
puppo        23
floofer       8
Name: dog_type, dtype: int64

#### 5) Changing unmeaningfull dog names to None
Were noticed clearly in Excel 

In [73]:
non_names = ['a', 'an', 'the', 'just', 'one', 'very', 'quite', 'not', 'actually', 
             'mad', 'space', 'infuriating', 'all', 'officially', '0', 'old', 'life',
             'unacceptable', 'my', 'incredibly', 'by', 'his', 'such']

for name in df_melted.name:
    if name in non_names:
        df_melted.name[df_melted.name == name] = 'None'

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys


In [75]:
df_melted['name'].unique()

array(['Biden', 'None', 'Cupcake', 'Reese', 'Philippe', 'Hall', 'Dook',
       'Fwed', 'Frank', 'Kial', 'George', 'Olive', 'Calvin', 'Naphaniel',
       'Genevieve', 'Joshwa', 'Jockson', 'Jeph', 'Carll', 'Klevin', 'Keet',
       'Dave', 'Clybe', 'Gabe', 'Pipsy', 'Bradlay', 'Churlie', 'Kenneth',
       'Clarence', 'Timison', 'Filup', 'Shaggy', 'Josep', 'Tedrick', 'Stu',
       'Erik', 'Cleopatricia', 'Otis', 'Jiminy', 'Alfie', 'Lugan',
       'Christoper', 'Walter', 'Scout', 'Kreggory', 'Jimothy', 'Phineas',
       'Cash', 'Nugget', 'Goose', 'Venti', 'Benedict', 'Nelly', 'Dante',
       'Penny', 'Aja', 'Emmy', 'Shadow', 'Beau', 'Derek', 'Mingus', 'Maya',
       'Canela', 'Jeffrey', 'Gerald', 'Ralphus', 'Zeke', 'Jim', 'Oliver',
       'Ted', 'Bruno', 'Koda', 'Zoey', 'Jax', 'Franklin', 'Darla',
       'Archie', 'Tilly', 'Waffles', 'Jimbo', 'Steven', 'Jack', 'Duddles',
       'Bailey', 'Romeo', 'Jesse', 'Bella', 'Louis', 'Elliot', 'Gary',
       'Rey', 'Koko', 'Alfy', 'Stanley', 'Rusty', '

#### 6) Investigate surprising values in rating_denominator

In [76]:
# List the different denominators
df_melted.rating_denominator.value_counts()

10     1954
50        3
80        2
11        2
170       1
150       1
120       1
110       1
90        1
70        1
40        1
20        1
7         1
2         1
Name: rating_denominator, dtype: int64

In [77]:

# Make available a column list for copying and pasting
df_melted.columns

Index(['tweet_id', 'timestamp', 'source', 'text', 'expanded_urls',
       'rating_numerator', 'rating_denominator', 'name', 'favorite_count',
       'retweet_count', 'jpg_url', 'img_num', 'p1', 'p1_conf', 'p1_dog', 'p2',
       'p2_conf', 'p2_dog', 'p3', 'p3_conf', 'p3_dog', 'dog_type'],
      dtype='object')

In [79]:
# Create a new dataframe with just tweet_id, text, rating_numerator and rating_denominator
rating_df = df_melted.drop(['timestamp','source','expanded_urls','name', 'favorite_count',
       'retweet_count', 'jpg_url', 'img_num', 'p1', 'p1_conf', 'p1_dog', 'p2',
       'p2_conf', 'p2_dog', 'p3', 'p3_conf', 'p3_dog', 'dog_type'],axis=1)

In [80]:
rating_df = rating_df[rating_df.rating_denominator != 10]

In [81]:
# Set the column width in pandas so the full tweet text can be read for analysis
pd.set_option("display.max_colwidth",280)

In [82]:
# Display full dataframe
rating_df

tweet_id  \
1950  666287406224695296   
3284  682962037429899265   
3259  684222868335505415   
2934  710658690886586372   
2952  709198395643068416   
2889  716439118184652801   
2910  713900603437621249   
3091  697463031882764288   
3016  704054845121142784   
3449  675853064436391936   
3391  677716515794329600   
2353  810984652412424192   
2291  820690176645140481   
2765  740373189193256964   
2629  758467244762497024   
2855  722974582966214656   
2813  731156023742988288   

                                                                                                                                                                           text  \
1950                                   This is an Albanian 3 1/2 legged  Episcopalian. Loves well-polished hardwood flooring. Penis on the collar. 9/10 https://t.co/d9NcXFKwLv   
3284                                This is Darrel. He just robbed a 7/11 and is in a high speed police chase. Was just spotted by the helicopter 10/10 https://t.co/7EsP8LmSp5   
3259                               Someone help the girl is being mugged. Several are distracting her while two steal her shoes. Clever puppers 121/110 https://t.co/1zfnTJLt55   
2934                                                               Here's a brigade of puppers. All look very prepared for whatever happens next. 80/80 https://t.co/0eb7R1Om12   
2952                         From left to right:\nCletus, Jerome, Alejandro, Burp, &amp; Titson\nNone know where camera is. 45/50 would hug all at once https://t.co/sedre1ivTK   
2889                                                      This is Bluebert. He just saw that both #FinalFur match ups are split 50/50. Amazed af. 11/10 https://t.co/Kky1DPG4iq   
2910                                                                              Happy Saturday here's 9 puppers on a bench. 99/90 good work everybody https://t.co/mpvaVxKmc1   
3091                                                                               Happy Wednesday here's a bucket of pups. 44/40 would pet all at once https://t.co/HppvrYuamZ   
3016                                                                                         Here is a whole flock of puppers.  60/50 I'll take the lot https://t.co/9dpcw6MdWa   
3449                                                                Here we have an entire platoon of puppers. Total score: 88/80 would pet all at once https://t.co/y93p6FLvVw   
3391                                                                                                     IT'S PUPPERGEDDON. Total of 144/120 ...I think https://t.co/ZanVtAtvIq   
2353  Meet Sam. She smiles 24/7 &amp; secretly aspires to be a reindeer. \nKeep Sam smiling by clicking and sharing this link:\nhttps://t.co/98tB8y7y7t https://t.co/LouL5vdvxx   
2291                                                                        The floofs have been released I repeat the floofs have been released. 84/70 https://t.co/NIYC820tmd   
2765                               After so many requests, this is Bretagne. She was the last surviving 9/11 search dog, and our second ever 14/10. RIP https://t.co/XAVDNDaVgQ   
2629                                                                                             Why does this never happen at my front door... 165/150 https://t.co/HmwrdfEfUE   
2855                                                                                                           Happy 4/20 from the squad! 13/10 for all https://t.co/eV1diwds8a   
2813                                                  Say hello to this unbelievably well behaved squad of doggos. 204/170 would try to pet all at once https://t.co/yGQI3He3xv   

      rating_numerator  rating_denominator  
1950                 1                   2  
3284                 7                  11  
3259               121                 110  
2934                80                  80  
2952                45                  50  
2889              

In [83]:
# Reading the text of Tweet_id 666287406224695296 - the correct value should be 9/10
# Change numerator and denominator rating scores in master dataframe df_melted
df_melted.loc[df_melted.tweet_id == 666287406224695296, 'rating_numerator'] = 9
df_melted.loc[df_melted.tweet_id == 666287406224695296, 'rating_denominator'] = 10

In [84]:
# Reading the text of Tweet_id 682962037429899265 - the correct value should be 10/10
# Change numerator and denominator rating scores in master dataframe df_melted
df_melted.loc[df_melted.tweet_id == 682962037429899265, 'rating_numerator'] = 10
df_melted.loc[df_melted.tweet_id == 682962037429899265, 'rating_denominator'] = 10

In [85]:
# Reading the text of Tweet_id 716439118184652801 - the correct value should be 11/10
# Change numerator and denominator rating scores in master dataframe df_melted
df_melted.loc[df_melted.tweet_id == 716439118184652801, 'rating_numerator'] = 11
df_melted.loc[df_melted.tweet_id == 716439118184652801, 'rating_denominator'] = 10

In [86]:
# Reading the text of Tweet_id 810984652412424192 - there is no correct value in the text so allocated 10/10
# so has to be neutral
# Change numerator and denominator rating scores in master dataframe df_melted
df_melted.loc[df_melted.tweet_id == 810984652412424192, 'rating_numerator'] = 10
df_melted.loc[df_melted.tweet_id == 810984652412424192, 'rating_denominator'] = 10

In [87]:
# Reading the text of Tweet_id 740373189193256964 - the correct value should be 14/10
# Change numerator and denominator rating scores in master dataframe df_melted
df_melted.loc[df_melted.tweet_id == 740373189193256964, 'rating_numerator'] = 14
df_melted.loc[df_melted.tweet_id == 740373189193256964, 'rating_denominator'] = 10

In [88]:
# Reading the text of Tweet_id 722974582966214656 - the correct value should be 13/10
# Change numerator and denominator rating scores in master dataframe df_melted
df_melted.loc[df_melted.tweet_id == 722974582966214656, 'rating_numerator'] = 13
df_melted.loc[df_melted.tweet_id == 722974582966214656, 'rating_denominator'] = 10

In [89]:
# The other strange denominator ratings are due to the tweet being based on groups of dogs and the tweeter
# has given a rating for each member of the group. I'll leave these ratings as it reflects the text and photo.

# check the above changes to the rating_denominator have occurred
df_melted.rating_denominator.value_counts()

10     1960
80        2
50        2
170       1
150       1
120       1
110       1
90        1
70        1
40        1
Name: rating_denominator, dtype: int64

#### 7) Fixing source column values to be more clear

In [91]:
# Use re library with findall function and extract real source from tags > ...... <
import re
df_melted['source'] = df_melted['source'].apply(lambda x: re.findall(r'>(.*)<', x)[0])

In [92]:
# Check extracted correct information
df_melted['source'].value_counts()

Twitter for iPhone    1932
Twitter Web Client      28
TweetDeck               11
Name: source, dtype: int64

#### 8) Removing unnecessary '+0000' from timestamp

In [94]:
#Strip +0000 from timestamp column
df_melted.timestamp = df_melted.timestamp.str.strip('+0000')

#### 9) Change timestamp data type to datetime

In [95]:
# Convert data type to datetime
df_melted.timestamp = pd.to_datetime(df_melted.timestamp)

In [97]:
# Check the changes have been made
df_melted.info()
df_melted.head(2)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1971 entries, 1895 to 6618
Data columns (total 22 columns):
tweet_id              1971 non-null int64
timestamp             1971 non-null datetime64[ns]
source                1971 non-null object
text                  1971 non-null object
expanded_urls         1971 non-null object
rating_numerator      1971 non-null int64
rating_denominator    1971 non-null int64
name                  1971 non-null object
favorite_count        987 non-null float64
retweet_count         987 non-null float64
jpg_url               1971 non-null object
img_num               1971 non-null float64
p1                    1971 non-null object
p1_conf               1971 non-null float64
p1_dog                1971 non-null object
p2                    1971 non-null object
p2_conf               1971 non-null float64
p2_dog                1971 non-null object
p3                    1971 non-null object
p3_conf               1971 non-null float64
p3_dog               

tweet_id           timestamp              source  \
1895  667405339315146752 2015-11-19 18:13:27  Twitter for iPhone   
1894  667435689202614272 2015-11-19 20:14:03  Twitter for iPhone   

                                                                   text  \
1895  This is Biden. Biden just tripped... 7/10 https://t.co/3Fm9PwLju1   
1894                            Ermergerd 12/10 https://t.co/PQni2sjPsm   

                                                        expanded_urls  \
1895  https://twitter.com/dog_rates/status/667405339315146752/photo/1   
1894  https://twitter.com/dog_rates/status/667435689202614272/photo/1   

      rating_numerator  rating_denominator   name  favorite_count  \
1895                 7                  10  Biden           489.0   
1894                12                  10   None             NaN   

      retweet_count    ...                p1   p1_conf p1_dog  \
1895          234.0    ...     Saint_Bernard  0.381377   True   
1894            NaN    ...        Rottweiler  0.999091   True   

                      p2   p2_conf p2_dog                       p3   p3_conf  \
1895            Leonberg  0.127998   True         golden_retriever  0.069357   
1894  miniature_pinscher  0.000450   True  black-and-tan_coonhound  0.000157   

     p3_dog  dog_type  
1895   True      None  
1894   True      None  

[2 rows x 22 columns]